In [37]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sb, joblib

In [38]:
# 데이터 불러오기
df = pd.read_csv("../data/고객db_전처리.csv")

In [39]:
# 차량 구매 서비스 플랫폼 -> 회원 정보, 구매 정보 포함
df

,연번,이름,생년월일,현재 나이,연령대,성별,휴대폰 번호,이메일,고객 구분,아이디,...,최근 구매 시점,최근 구매 요일,마지막 구매 이후 경과 기간,최근 구매 당시 나이,최근 거래 금액,선호 거래 방식,선호 구매 경로,차량 구매 횟수,신차 구매 지수,브랜드 충성도 지수
0,1,강주온,2002-03-04,23,20대 중반,남,010-6502-8753,janisi@himedia.co.kr,법인,14798,...,2024 2분기,평일,350,22,34490000,신용카드,오프라인,1,0.002,0.135
1,2,김경수,1983-01-02,42,40대 초반,남,010-1111-2222,kimkyoungsoo31@himedia.co.kr,법인,kyoungsoo031,...,2025 1분기,평일,38,42,86600000,신용카드,오프라인,5,0.508,0.037
2,3,김다은,1999-05-02,26,20대 중반,여,010-6789-0123,kimdaeun26@himedia.co.kr,법인,daeun026,...,2025 1분기,평일,10,26,69590000,계좌이체,오프라인,1,0.161,0.004
3,4,김민수,1960-07-01,65,60대 중반,남,010-1234-5678,kimminsu1@himedia.co.kr,법인,minsu001,...,2023 2분기,평일,689,63,52260000,현금,오프라인,1,0.002,0.130
4,5,김승현,1970-03-02,55,50대 중반,남,010-0123-4567,kimseunghyun20@himedia.co.kr,법인,seunghyun020,...,2023 4분기,주말,536,53,46840000,신용카드,오프라인,1,0.004,0.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,288,신시우,1958-07-11,67,60대 후반,여,042-207-6955,gyeonghyi78@gmail.com,개인,jihungim,...,2024 4분기,평일,90,66,83140000,신용카드,온라인,2,0.011,0.280
287,289,최춘자,1976-06-27,49,40대 후반,여,031-170-7739,hancunja@yu.kr,개인,jinho50,...,2023 3분기,평일,574,47,37460000,현금,오프라인,3,0.047,0.116
288,290,차서준,1968-01-22,57,50대 후반,여,064-741-3740,qgim@simryujo.com,개인,jihun41,...,2024 1분기,평일,433,56,71820000,계좌이체,온라인,2,0.005,0.304
289,291,안정희,1984-01-10,41,40대 초반,남,051-406-1803,donghyeonbae@joyunson.com,개인,yeongil47,...,2024 1분기,평일,415,40,88190000,현금,온라인,2,0.006,0.268


# 예측 모델 개발

In [40]:
# 전처리 기본 세팅
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [41]:
df

,연번,이름,생년월일,현재 나이,연령대,성별,휴대폰 번호,이메일,고객 구분,아이디,...,최근 구매 시점,최근 구매 요일,마지막 구매 이후 경과 기간,최근 구매 당시 나이,최근 거래 금액,선호 거래 방식,선호 구매 경로,차량 구매 횟수,신차 구매 지수,브랜드 충성도 지수
0,1,강주온,2002-03-04,23,20대 중반,남,010-6502-8753,janisi@himedia.co.kr,법인,14798,...,2024 2분기,평일,350,22,34490000,신용카드,오프라인,1,0.002,0.135
1,2,김경수,1983-01-02,42,40대 초반,남,010-1111-2222,kimkyoungsoo31@himedia.co.kr,법인,kyoungsoo031,...,2025 1분기,평일,38,42,86600000,신용카드,오프라인,5,0.508,0.037
2,3,김다은,1999-05-02,26,20대 중반,여,010-6789-0123,kimdaeun26@himedia.co.kr,법인,daeun026,...,2025 1분기,평일,10,26,69590000,계좌이체,오프라인,1,0.161,0.004
3,4,김민수,1960-07-01,65,60대 중반,남,010-1234-5678,kimminsu1@himedia.co.kr,법인,minsu001,...,2023 2분기,평일,689,63,52260000,현금,오프라인,1,0.002,0.130
4,5,김승현,1970-03-02,55,50대 중반,남,010-0123-4567,kimseunghyun20@himedia.co.kr,법인,seunghyun020,...,2023 4분기,주말,536,53,46840000,신용카드,오프라인,1,0.004,0.103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,288,신시우,1958-07-11,67,60대 후반,여,042-207-6955,gyeonghyi78@gmail.com,개인,jihungim,...,2024 4분기,평일,90,66,83140000,신용카드,온라인,2,0.011,0.280
287,289,최춘자,1976-06-27,49,40대 후반,여,031-170-7739,hancunja@yu.kr,개인,jinho50,...,2023 3분기,평일,574,47,37460000,현금,오프라인,3,0.047,0.116
288,290,차서준,1968-01-22,57,50대 후반,여,064-741-3740,qgim@simryujo.com,개인,jihun41,...,2024 1분기,평일,433,56,71820000,계좌이체,온라인,2,0.005,0.304
289,291,안정희,1984-01-10,41,40대 초반,남,051-406-1803,donghyeonbae@joyunson.com,개인,yeongil47,...,2024 1분기,평일,415,40,88190000,현금,온라인,2,0.006,0.268


In [42]:
# X, y 분할
X = df[["최근 거래 금액", "거주 지역", "차량 사이즈", "차량 유형", "연료 구분"]]
y = df["최근 구매 제품"]

In [43]:
# X = 최근 거래 금액, 거주 지역, 차량 사이즈, 차량 유형, 연료 구분
# y = 최근 구매 제품

In [44]:
X

,최근 거래 금액,거주 지역,차량 사이즈,차량 유형,연료 구분
0,34490000,인천광역시,준중형,세단,휘발유
1,86600000,광주광역시,대형,세단,휘발유
2,69590000,부산광역시,중형,SUV,수소
3,52260000,전라남도 목포시,중형,SUV,디젤
4,46840000,경기도 수원시,중형,세단,휘발유
...,...,...,...,...,...
286,83140000,전라북도 전주시,대형,세단,휘발유
287,37460000,충청남도 천안시,준중형,세단,휘발유
288,71820000,경상남도 창원시,중형,SUV,수소
289,88190000,대구광역시,대형,세단,휘발유


In [45]:
y

0      Avante (CN7 N)
1           G80 (RG3)
2           NEXO (FE)
3          Santa-Fe ™
4            G70 (IK)
            ...      
286         G80 (RG3)
287    Avante (CN7 N)
288         NEXO (FE)
289         G80 (RG3)
290         G80 (RG3)
Name: 최근 구매 제품, Length: 291, dtype: object

In [46]:
# 인코더 생성
label = LabelEncoder()
onehot = OneHotEncoder()

In [47]:
X = pd.get_dummies(X, columns=["거주 지역", "차량 사이즈", "차량 유형", "연료 구분"])

In [48]:
X

,최근 거래 금액,거주 지역_경기도 성남시,거주 지역_경기도 수원시,거주 지역_경상남도 창원시,거주 지역_경상북도 포항시,거주 지역_광주광역시,거주 지역_대구광역시,거주 지역_대전광역시,거주 지역_부산광역시,거주 지역_서울특별시,...,차량 사이즈_프리미엄,차량 유형_SUV,차량 유형_세단,차량 유형_해치백,연료 구분_디젤,연료 구분_수소,연료 구분_전기,연료 구분_플러그인 하이브리드,연료 구분_하이브리드,연료 구분_휘발유
0,34490000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
1,86600000,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
2,69590000,False,False,False,False,False,False,False,True,False,...,False,True,False,False,False,True,False,False,False,False
3,52260000,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
4,46840000,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,83140000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
287,37460000,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
288,71820000,False,False,True,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
289,88190000,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True


In [49]:
y

0      Avante (CN7 N)
1           G80 (RG3)
2           NEXO (FE)
3          Santa-Fe ™
4            G70 (IK)
            ...      
286         G80 (RG3)
287    Avante (CN7 N)
288         NEXO (FE)
289         G80 (RG3)
290         G80 (RG3)
Name: 최근 구매 제품, Length: 291, dtype: object

# 데이터셋 분할

In [50]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# 고객 추천 모델 --> Classification
# GBM, Linear SVM, CatBoost, KNN, GaussianNB, MultinomialNB, BernoulliNB, MLP 필요
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [52]:
lr = LogisticRegression()
svc = SVC()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()
gnb = GaussianNB()
knn = KNeighborsClassifier()
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
lgb = LGBMClassifier()
cb = CatBoostClassifier()

In [53]:
lr.fit(train_X, train_y)
svc.fit(train_X, train_y)
dtc.fit(train_X, train_y)
rfc.fit(train_X, train_y)
gbc.fit(train_X, train_y)
gnb.fit(train_X, train_y)
knn.fit(train_X, train_y)
lgb.fit(train_X, train_y)
cb.fit(train_X, train_y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 98
[LightGBM] [Info] Number of data points in the train set: 232, number of used features: 11
[LightGBM] [Info] Start training from score -1.757858
[LightGBM] [Info] Start training from score -1.640075
[LightGBM] [Info] Start training from score -1.835819
[LightGBM] [Info] Start training from score -5.446737
[LightGBM] [Info] Start training from score -3.500827
[LightGBM] [Info] Start training from score -4.348125
[LightGBM] [Info] Start training from score -5.446737
[LightGBM] [Info] Start training from score -1.685537
[LightGBM] [Info] Start training from score -4.348125
[LightGBM] [Info] Start training from score -5.446737
[LightGBM] [Info] Start training from score -1.596590
[LightGBM] [Info] Start training from sco

In [54]:
y_pred_lr = lr.predict(test_X)
y_pred_svc = svc.predict(test_X)
y_pred_dtc = dtc.predict(test_X)
y_pred_rfc = rfc.predict(test_X)
y_pred_gbc = gbc.predict(test_X)
y_pred_gnb = gnb.predict(test_X)
y_pred_knn = knn.predict(test_X)
y_pred_lgb = lgb.predict(test_X)
y_pred_cb = cb.predict(test_X)

In [55]:
# 정확도 확인
print("Logistic Regression 정확도 : ", accuracy_score(test_y, y_pred_lr))
print("SVM 정확도 : ", accuracy_score(test_y, y_pred_svc))
print("Decision Tree 정확도 : ", accuracy_score(test_y, y_pred_dtc))
print("Random Forest 정확도 : ", accuracy_score(test_y, y_pred_rfc))
print("Gradient Boosting 정확도 : ", accuracy_score(test_y, y_pred_gbc))
print("Gaussian Naive Bayes 정확도 : ", accuracy_score(test_y, y_pred_gnb))
print("KNN 정확도 : ", accuracy_score(test_y, y_pred_knn))
print("LightGBM 정확도 : ", accuracy_score(test_y, y_pred_lgb))
print("CatBoost 정확도 : ", accuracy_score(test_y, y_pred_cb))

Logistic Regression 정확도 :  0.15254237288135594
SVM 정확도 :  0.6101694915254238
Decision Tree 정확도 :  0.9661016949152542
Random Forest 정확도 :  0.9830508474576272
Gradient Boosting 정확도 :  0.9830508474576272
Gaussian Naive Bayes 정확도 :  0.8135593220338984
KNN 정확도 :  0.7288135593220338
LightGBM 정확도 :  0.9830508474576272
CatBoost 정확도 :  0.9830508474576272


In [56]:
joblib.dump(dtc, "models/DecisionTree 모델.pkl")
joblib.dump(rfc, "models/RandomForest 모델.pkl")
joblib.dump(gbc, "models/GradientBoosting 모델.pkl")
joblib.dump(lgb, "models/LightGBM 모델.pkl")
joblib.dump(cb, "models/CatBoost 모델.pkl")

['models/CatBoost 모델.pkl']

### dtc, rfc, gbc, lgb, cb 모델의 정확도가 높음

In [57]:
test_X

,최근 거래 금액,거주 지역_경기도 성남시,거주 지역_경기도 수원시,거주 지역_경상남도 창원시,거주 지역_경상북도 포항시,거주 지역_광주광역시,거주 지역_대구광역시,거주 지역_대전광역시,거주 지역_부산광역시,거주 지역_서울특별시,...,차량 사이즈_프리미엄,차량 유형_SUV,차량 유형_세단,차량 유형_해치백,연료 구분_디젤,연료 구분_수소,연료 구분_전기,연료 구분_플러그인 하이브리드,연료 구분_하이브리드,연료 구분_휘발유
84,87930000,False,False,False,False,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
259,86420000,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
45,37900000,False,False,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,True
176,45570000,False,False,False,False,False,False,False,False,True,...,False,False,True,False,False,False,False,False,False,True
143,34920000,False,True,False,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
125,44160000,False,False,False,False,False,False,False,False,True,...,False,True,False,False,True,False,False,False,False,False
226,69810000,True,False,False,False,False,False,False,False,False,...,False,True,False,False,False,True,False,False,False,False
118,51220000,False,True,False,False,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
9,129780000,False,False,True,False,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,True
90,44940000,False,False,False,False,False,False,False,False,True,...,False,True,False,False,True,False,False,False,False,False


In [58]:
new_data = np.array([35000000, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0,0,0,1,0, 1,0,0, 0,0,0,0,0,1]).reshape(1, 30)

In [59]:
recom_list = []

In [60]:
recom_list.append(dtc.predict(new_data)[0])
recom_list.append(rfc.predict(new_data)[0])
recom_list.append(gbc.predict(new_data)[0])
recom_list.append(lgb.predict(new_data)[0])
recom_list.append(cb.predict(new_data)[0][0])

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [61]:
recom_list

['G70 (IK)', 'G70 (IK)', 'G70 (IK)', 'G70 (IK)', 'Santa-Fe ™']

In [62]:
recom_list = list(set(recom_list))

In [63]:
# 추천 결과 하나씩 보여주기
for i in recom_list :
    print(i)

G70 (IK)
Santa-Fe ™


# 비슷한 가격대 전기차 3대 추천

In [64]:
elec_car_compen = {
    "서울특별시": 9000000,
    "부산광역시": 10500000,
    "대구광역시": 11000000,
    "인천광역시": 10600000,
    "광주광역시": 11000000,
    "대전광역시": 12000000,
    "울산광역시": 10500000,
    "경기도 수원시": 10500000,
    "경기도 성남시": 11000000,
    "충청북도 청주시": 14000000,
    "충청남도 천안시": 14000000,
    "전라북도 전주시": 15000000,
    "전라남도 목포시": 15500000,
    "경상북도 포항시": 13000000,
    "경상남도 창원시": 13000000
}

if "연료 구분" in ["전기", "플러그인 하이브리드", "하이브리드"]:
     pass
else: # 추천 시작
     compen = elec_car_compen["서울특별시"]
     # df.loc[df["최근 거래 금액"] <= 45000000 + compen, "최근 구매 제품"].to_numpy()[0].head(3)

In [65]:
compen = elec_car_compen["서울특별시"]
recom_elec = df.loc[(df["최근 거래 금액"] <= 45000000 + compen) & (df["연료 구분"].isin(["전기", "플러그인 하이브리드", "하이브리드"])), "최근 구매 제품"].to_numpy()[0:3]

In [66]:
for i in recom_elec:
    print(i)

IONIQ 6 (CE)
Santa-Fe (MX5 PHEV)
Grandeur (GN7 HEV)


# 프로모션 : 신용카드 혜택 설명하면서 50% 비싼 차 추천 -> 옵션 붙인 걸로 생각하면 됨

In [31]:
prom_data = np.array([35000000 * 1.5, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0,0,0,1,0, 1,0,0, 0,0,0,0,0,1]).reshape(1, 30)

In [32]:
prom_list = []

In [33]:
prom_list.append(dtc.predict(prom_data)[0])
prom_list.append(rfc.predict(prom_data)[0])
prom_list.append(gbc.predict(prom_data)[0])
prom_list.append(lgb.predict(prom_data)[0])
prom_list.append(cb.predict(prom_data)[0][0])

/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/anaconda3/envs/hi/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [34]:
prom_list = list(set(prom_list))

In [35]:
prom_list

['G70 (IK)', 'NEXO (FE)']

In [36]:
# 프로모션 결과와 추천 결과 비교해서, 같은 결과는 제외하고 보여주기
# 모두 다 같으면 애초에 프로모션 안 함
for i in prom_list :
    if i not in recom_list :
        print(i)

NEXO (FE)
